In [3]:
import pandas as pd
import numpy as np
import re
import pymupdf

from PyPDF2 import PdfReader

# Leitura dos Microdados

In [4]:
# df = pd.read_csv('microdados_enem_2018/DADOS/MICRODADOS_ENEM_2018.csv', sep=';', encoding='latin-1', nrows=1000000)
# df.head()

**2018**

In [ ]:
df_2018 = pd.read_csv('microdados_enem_2018/DADOS/ITENS_PROVA_2018.csv', sep=';', encoding='latin-1')

df_ch_2018 = df_2018.loc[df_2018['SG_AREA'] == 'CH']

df_ch_2018 = df_ch_2018.loc[df_ch_2018['TX_COR']=='Azul'].loc[df_ch_2018['CO_PROVA']==451][['CO_POSICAO', 'TX_GABARITO', 'NU_PARAM_A','NU_PARAM_B',	'NU_PARAM_C']]
df_ch_2018['ANO'] = '2018'
df_ch_2018.head()

,CO_POSICAO,TX_GABARITO,NU_PARAM_A,NU_PARAM_B,NU_PARAM_C,ANO
501,46,C,5.10044,1.51100,0.13957,2018
516,47,A,3.21792,1.65990,0.14834,2018
527,48,C,3.46765,1.30736,0.15603,2018
536,49,D,2.67992,1.47683,0.11389,2018
547,50,D,4.00369,1.39940,0.16074,2018


**2019**


In [ ]:
df_2019 = pd.read_csv('microdados_enem_2019/DADOS/ITENS_PROVA_2019.csv', sep=';', encoding='latin-1')

df_ch_2019 = df_2019.loc[df_2019['SG_AREA'] == 'CH']
df_ch_2019

df_ch_2019 = df_ch_2019.loc[df_ch_2019['TX_COR']=='Azul'].loc[df_ch_2019['CO_PROVA']==507][['CO_POSICAO', 'TX_GABARITO', 'NU_PARAM_A','NU_PARAM_B',	'NU_PARAM_C']]
df_ch_2019['ANO'] = '2019'
df_ch_2019.head()

,CO_POSICAO,TX_GABARITO,NU_PARAM_A,NU_PARAM_B,NU_PARAM_C,ANO
559,46,B,2.21881,1.10809,0.13721,2019
570,47,A,2.07127,1.34720,0.07736,2019
582,48,C,NaN,NaN,NaN,2019
585,49,C,1.31749,0.44063,0.18259,2019
599,50,C,3.45284,1.65017,0.09484,2019


**2020**

In [ ]:
df_2020 = pd.read_csv('microdados_enem_2020/DADOS/ITENS_PROVA_2020.csv', sep=';', encoding='latin-1')

df_ch_2020 = df_2020.loc[df_2020['SG_AREA'] == 'CH']

df_ch_2020 = df_ch_2020.loc[df_ch_2020['TX_COR']=='AZUL'].loc[df_ch_2020['CO_PROVA']==567][['CO_POSICAO', 'TX_GABARITO', 'NU_PARAM_A','NU_PARAM_B',	'NU_PARAM_C']]
df_ch_2020['ANO'] = '2020'
df_ch_2020 = df_ch_2020.sort_values(by=['CO_POSICAO'])
df_ch_2020.head()

,CO_POSICAO,TX_GABARITO,NU_PARAM_A,NU_PARAM_B,NU_PARAM_C,ANO
4141,46,D,3.53336,1.34087,0.13677,2020
4113,47,A,2.47767,1.82861,0.30933,2020
3787,48,B,2.15838,1.83008,0.13413,2020
1239,49,C,3.11394,1.33811,0.15766,2020
2447,50,B,1.33904,-0.24666,0.20167,2020


# Leituras da Provas

In [10]:
# Caminho da prova ENEM 2018
prova_2018 = 'provas/2018_PV_impresso_D1_CD1.pdf'

# Caminho da prova ENEM 2019
prova_2019 = 'provas/2019_PV_impresso_D1_CD1.pdf'

# Caminho da prova ENEM 2020
prova_2020 = 'provas/2020_PV_impresso_D1_CD1.pdf'

In [ ]:
# Função que recebe o caminho do pdf, a página de início e a de fim e retorna as questões
def get_questions(pdf_path: str, start_page: int, end_page: int, caps_lock: bool = False):
    def extract_questions(full_text: str):
        aux = full_text.split('QUESTÃO ') if caps_lock else full_text.split('Questão ')
        questions = [question for question in aux if question[:2].isnumeric()]
        final_questions = [question[3:] for question in questions]
        return final_questions

    reader = PdfReader(pdf_path)

    final_questions = []
    for i in range(start_page, end_page):
        full_text = reader.pages[i].extract_text()
        questions = extract_questions(full_text)
        final_questions.extend(questions)

    return final_questions

In [82]:
questoes_2018 = get_questions(prova_2018, 18, 32, True)
len(questoes_2018)

45

In [83]:
questoes_2019 = get_questions(prova_2019, 20, 32)
len(questoes_2019)

45

In [84]:
questoes_2020 = get_questions(prova_2020, 19, 32)
len(questoes_2020)

45

## Alternativas


In [ ]:
# Função que retorna as questões sem as alternativas e as alternativas formatadas
def get_alternatives(questions: list[str]) -> tuple[list[str], list[str]]:
    # Função auxiliar que formata as quesões em formato de dicionário
    def format_alt(alt: str) -> str:
        search = re.search(r"(A\s.+?)(B\s.+?)(C\s.+?)(D\s.+?)(E\s.+)", alt)
        if search:
            groups = search.groups()
            alt = '; '.join([f"{item[0]}: {item[2:-1]}" for item in groups])
        else:
            alt = ''
        return alt

    # Pra cada questão separa a alternativa e a formata
    alts = []
    questions_new = []
    for question in questions:
        full_question = re.split(r"(\nA\s.*\n)", question)

        alt = ''.join(full_question[-2:])
        alt = alt.replace('\n', '')
        alt = re.sub(r"\*.*\*", "", alt)
        alt = re.sub(r"(CH).*\d", "", alt)
        alt = format_alt(alt)
        
        question_new = ''.join(full_question[:-2])
        question_new = question_new.replace('\n', "")

        alts.append(alt)
        questions_new.append(question_new)

    return questions_new, alts

**2018**

In [86]:
questoes_2018, alts_2018 = get_alternatives(questoes_2018)
alts_2018[:5]

['A: preservação cultural dos territórios ocupados; B: formação humanitária da sociedade europeia; C: catalogação de dados úteis aos propósitos colonialistas; D: desenvolvimento de técnicas matemáticas de construção de cartas; E: consolidação do conhecimento topográfico como campo acadêmico',
 'A: surgimento do monasticismo guerreiro, ocasionado pelas cruzadas; B: descentralização do poder eclesiástico, produzida pelo feudalismo; C: alastramento da peste bubônica, provocado pela expansão comercial; D: afirmação da fraternidade mendicante, estimulada pela reforma espiritual; E: criação das faculdades de medicina, promovida pelo renascimento urbano',
 'A: Imposição ideológica e normas hierárquicas; B: Determinação divina e soberania monárquica; C: Intervenção consensual e autonomia comunitária; D: Mediação jurídica e regras contratualistas; E: Gestão coletiva e obrigações tributárias',
 'A: reunir os antagonismos das opiniões ao método dialético; B: ajustar a clareza do conhecimento ao i

**2019**

In [87]:
questoes_2019, alts_2019 = get_alternatives(questoes_2019)
alts_2019[:5]

['A: concepção apresentada considera a existência de uma nova época geológica concebida a partir da capacidade de influência humana nos processosA eruptivos; B: exógenos; C: tectônicos; D: magmáticos; E: metamórficos',
 'A: Albedo dos corpos físicos; B: Profundidade do lençol freático; C: Campo de magnetismo terrestre; D: Qualidade dos recursos minerais; E: Movimento de translaç ão planetária',
 'A: relação entre ser humano e natureza ressaltada no texto refletia a permanência da seguinte corrente filosófica:A Relativismo cognitivo; B: Materialismo dialético; C: Racionalismo cartesiano; D: Pluralismo epistemológico; E: Existencialismo fenomenológico',
 'A: Preservação da área de mata ciliar; B: Adoção da prática de adubação química; C: Utilização da técnica de controle biológico; D: Ampliação do modelo de monocultura tropical; E: Intensificação da drenagem do solo de várzea.SatéliteSolEnergia solarrefletidaEnergia solar incidenteFloresta Rio Pastagens RodoviaAtmosfer',
 'A: anulação da

**2020**

In [88]:
questoes_2020, alts_2020 = get_alternatives(questoes_2020)
alts_2020[:5]

['A:  expansão do setor extrativista; B:  incremento da atividade agrícola; C:  diversificação da matriz energética; D:  fortalecimento da pesquisa científica; E:  monitoramento do fluxo alfandegário',
 'A:  heterogeneidade do modo de vida agrário; B:  redução do fluxo populacional nas cidades; C:  correlação entre força de trabalho e migração sazonal; D:  indissociabilidade entre local de moradia e acesso à renda; E:  desregulamentação das propriedades nas zonas de fronteira',
 'A:  estagnação de direitos adquiridos e do anacronismo da legislação vigente; B:  manutenção do status quo gerencial e dos padrões de socialização familiar; C:  desestruturação da herança patriarcal e das mudanças do perfil ocupacional; D:  disputas na composição sindical e da presença na esfera político-partidária; E:  exigências de aperfeiçoamento profissional e de habilidades na competência diretiva',
 'A:  reverter o processo de privatização fundiária; B:  ressaltar a inviabilidade da produção latifundiári

# Montando o dataset final

In [ ]:
# Função auxiliar para criar o dataset com as novas colunas
def merge_quests_alts(dataset: pd.DataFrame, questions: list[str], alts: list[str]) -> pd.DataFrame:
    dataset['QUESTOES'] = questions
    dataset['ALTERNATIVAS'] = alts

    return dataset

In [98]:
df_final = pd.concat([merge_quests_alts(df_ch_2018, questoes_2018, alts_2018),
                      merge_quests_alts(df_ch_2019, questoes_2019, alts_2019),
                      merge_quests_alts(df_ch_2020, questoes_2020, alts_2020)], axis=0)

df_final

,CO_POSICAO,TX_GABARITO,NU_PARAM_A,NU_PARAM_B,NU_PARAM_C,ANO,QUESTOES,ALTERNATIVAS
501,46,C,5.10044,1.51100,0.13957,2018,No Segundo Congresso Internacional de Ciências...,A: preservação cultural dos territórios ocupad...
516,47,A,3.21792,1.65990,0.14834,2018,A existência em Jerusalém de um hospital volta...,"A: surgimento do monasticismo guerreiro, ocasi..."
527,48,C,3.46765,1.30736,0.15603,2018,"A tribo não possui um rei, mas um chefe que nã...",A: Imposição ideológica e normas hierárquicas;...
536,49,D,2.67992,1.47683,0.11389,2018,O filósofo reconhece-se pela posse inseparável...,A: reunir os antagonismos das opiniões ao méto...
547,50,D,4.00369,1.39940,0.16074,2018,Figura 1Disponível em: www.thehenryford.org. A...,A: impacto do medo da corrida armamentista.; B...
...,...,...,...,...,...,...,...,...
1302,86,D,3.55293,1.23410,0.18404,2020,Declaração de Direitos do Homem e do ...,A: manutenção das terras comunais; B: supres...
2674,87,E,3.71752,1.25879,0.16922,2020,"Na Grécia, o conceito de povo abrange...",A: competição para a escolha de representante...
4134,88,C,1.02685,0.10876,0.19583,2020,"TEXTO I Rio Tietê, São Paulo (SP). Fo...",A: valorização do sítio urbano; B: extinção ...
3279,89,D,2.78486,0.55811,0.19808,2020,"Adão, ainda que supuséssemos que suas...",A: A potência inata da mente.; B: A revelaçã...
